In [40]:
import pandas as pd
import re
import numpy as np
import math

In [2]:
input_data = pd.read_csv("data/test_input.csv", sep=",")

column_names = input_data.columns.tolist()
column_names[0] = "File"
input_data.columns = column_names

In [3]:
format_header = []
variables = ""
for x in input_data.columns.tolist():
    if "/" in x:
        m = re.search(r"([A-Za-z0-9]+[+-]){2,5} \|", x)
        column_val = m.group(0)
        format_header.append(column_val.replace(r" |", ""))
        variables = column_val.replace(r" |", "")
    else:
        format_header.append(x)
input_data.columns = format_header

In [4]:
input_data = input_data.iloc[:,:-1]
input_data.head

<bound method NDFrame.head of                        File    TUBE NAME    Group  CD107a+IFNg+MIP1b+TNFa+  \
0  T244_CD89 X-link_C03.fcs  CD89 X-link    Xlink                        0   
1  T244_Neg Control_C01.fcs  Neg Control  Control                        0   
2   W72_CD89 X-link_B03.fcs  CD89 X-link    Xlink                        0   
3   W72_Neg Control_B01.fcs  Neg Control  Control                        0   
4   V07_CD89 X-link_A03.fcs  CD89 X-link    Xlink                        0   
5   V07_Neg Control_A01.fcs  Neg Control  Control                        0   

   CD107a+IFNg+MIP1b+TNFa-  CD107a+IFNg+MIP1b-TNFa+  CD107a+IFNg+MIP1b-TNFa-  \
0                        0                        0                     0.00   
1                        0                        0                     0.00   
2                        0                        0                     0.00   
3                        0                        0                     0.38   
4                      

In [17]:
row_one = input_data.loc[1]
row_one

File                       T244_Neg Control_C01.fcs
TUBE NAME                               Neg Control
Group                                       Control
CD107a+IFNg+MIP1b+TNFa+                           0
CD107a+IFNg+MIP1b+TNFa-                           0
CD107a+IFNg+MIP1b-TNFa+                           0
CD107a+IFNg+MIP1b-TNFa-                         0.0
CD107a+IFNg-MIP1b+TNFa+                           0
CD107a+IFNg-MIP1b+TNFa-                           0
CD107a+IFNg-MIP1b-TNFa+                           0
CD107a+IFNg-MIP1b-TNFa-                        2.51
CD107a-IFNg+MIP1b+TNFa+                           0
CD107a-IFNg+MIP1b+TNFa-                           0
CD107a-IFNg+MIP1b-TNFa+                           0
CD107a-IFNg+MIP1b-TNFa-                         0.0
CD107a-IFNg-MIP1b+TNFa+                           0
CD107a-IFNg-MIP1b+TNFa-                           0
CD107a-IFNg-MIP1b-TNFa+                         0.0
Name: 1, dtype: object

In [59]:
final_df = pd.DataFrame()
for j in range(0, len(input_data.index)):
    row_one = input_data.loc[j]
    output_dict = {}
    output_df = pd.DataFrame()
    for i in range(0, len(row_one)):
        if "+" in row_one.index[i] or "-" in row_one.index[i]:
            searches = re.findall("[A-Za-z0-9]+[+-]", row_one.index[i])
            responses = [x[:-1] for x in searches]
            condition = [x[-1] for x in searches]
            value = row_one.values[i]
            output_dict.update( dict( zip( responses, condition ) ) )
            for resp, item in zip(responses, condition):
                output_df.at[i, resp] = item
            output_dict['value'] = value
            output_df.at[i, 'value'] = value
        else:
            output_df.at[i, row_one.index[i]] = row_one.values[i]
    output_df.iloc[:, 0] = [x for x in output_df.iloc[:, 0] if pd.notnull(x)] * len(output_df.index)
    output_df.iloc[:, 1] = [x for x in output_df.iloc[:, 1] if pd.notnull(x)] * len(output_df.index)
    output_df.iloc[:, 2] = [x for x in output_df.iloc[:, 2] if pd.notnull(x)] * len(output_df.index)
    output_df = output_df.dropna()
    final_df = pd.concat([final_df, output_df])
print(final_df)   
        
final_df.to_csv("~/Desktop/output.txt", sep="\t", index=False)
        

                        File    TUBE NAME    Group CD107a IFNg MIP1b TNFa  \
3   T244_CD89 X-link_C03.fcs  CD89 X-link    Xlink      +    +     +    +   
4   T244_CD89 X-link_C03.fcs  CD89 X-link    Xlink      +    +     +    -   
5   T244_CD89 X-link_C03.fcs  CD89 X-link    Xlink      +    +     -    +   
6   T244_CD89 X-link_C03.fcs  CD89 X-link    Xlink      +    +     -    -   
7   T244_CD89 X-link_C03.fcs  CD89 X-link    Xlink      +    -     +    +   
..                       ...          ...      ...    ...  ...   ...  ...   
13   V07_Neg Control_A01.fcs  Neg Control  Control      -    +     -    +   
14   V07_Neg Control_A01.fcs  Neg Control  Control      -    +     -    -   
15   V07_Neg Control_A01.fcs  Neg Control  Control      -    -     +    +   
16   V07_Neg Control_A01.fcs  Neg Control  Control      -    -     +    -   
17   V07_Neg Control_A01.fcs  Neg Control  Control      -    -     -    +   

    value  
3    0.00  
4    0.00  
5    0.00  
6    0.00  
7    0.00  
.. 

In [ ]:
        searches = re.findall("[A-Za-z0-9]+[+-]", row_one.index[i])
        responses = [x[:-1] for x in searches]
        condition = [x[-1] for x in searches]
        value = row_one.values[i]
        output_dict.update( dict( zip( responses, condition ) ) )
        output_dict['value'] = value